# <span style="color:teal">RBFE Network Setup</span>



In [ ]:
# import libraries
import BioSimSpace as BSS
import os
import glob
import csv
import numpy as np
import networkx as nx
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from sklearn.preprocessing import minmax_scale
import itertools

In [ ]:
# define all the folder locations
protein = "tyk2"
main_folder = "/home/anna/Documents/benchmark"

# for sem perturbations
tgt_to_run = f"{protein}_rename" #f"{protein}_me" f"{protein}_rename" for tyk2 and p38
cats_files_path = f"{main_folder}/scripts/RBFENN/ANALYSIS/perturbation_networks/output/series_predictions"

# other folders
system_folder = f"{main_folder}/{protein}_benchmark"
scripts_folder = f"{main_folder}/scripts" # scripts should be located in here
input_dir = f"{main_folder}/inputs"
path_to_ligands = f"{input_dir}/{protein}/ligands"
exec_folder = f"{system_folder}/execution_model_not_ejm53"

folder_list = [system_folder, exec_folder]
for fold in folder_list:
    if not os.path.isdir(fold):
        os.mkdir(fold)
        print(f"Made dir {fold}")

In [ ]:
# BSS.Notebook.View(f"{input_dir}/{protein}/protein/{protein}_parameterised.pdb").system()
#parameterise protein in advance

#### <span style="color:teal">2. Setting up the Network</span>
<a id="setup"></a>

##### <span style="color:teal">Choosing the parameters for the FEP runs</span>
<a id="parameters"></a>


In [ ]:
#nodes to pick things
node = BSS.Gateway.Node("A node to create input files for molecular dynamics simulation.")

node.addInput("Ligand FF", BSS.Gateway.String(help="Force field to parameterise ligands with.",
                                             allowed=["GAFF2", "Parsely", "Sage"],
                                             default="Sage"))

node.addInput("Protein FF", BSS.Gateway.String(help="Force field to parameterise the protein with.",
                                             allowed=["FF03", "FF14SB", "FF99", "FF99SB", "FF99SBILDN"],
                                             default="FF14SB"))

node.addInput("Water Model", BSS.Gateway.String(help="Water model to use.",
                                             allowed=["SPC", "SPCE", "TIP3P", "TIP4P", "TIP5P"],
                                             default="TIP3P"))

node.addInput("Box Edges", BSS.Gateway.String(help="Size of water box around molecular system.",
                                             allowed=["20*angstrom", "25*angstrom", "30*angstrom", "35*angstrom", "45*angstrom", "5*nm", "7*nm", "10*nm"],
                                             default="30*angstrom"))

node.addInput("Box Shape", BSS.Gateway.String(help="Geometric shape of water box.",
                                             allowed=["cubic", "truncatedOctahedron"],
                                             default="truncatedOctahedron"))

node.addInput("Run Time", BSS.Gateway.String(help="The sampling time per lambda window.",
                                             allowed=["10*ps", "100*ps", "1*ns", "2*ns", "3*ns", "4*ns", "5*ns", "8*ns", "10*ns", "12*ns", "15*ns"],
                                             default="4*ns"))

node.addInput("HMR", BSS.Gateway.String(help="Whether or not Hydrogen Mass repartitioning should be used. If true, a timestep of 4 fs will be used.",
                                             allowed=["True","False"],
                                             default="True"))

engines_options = [e.upper() for e in BSS.FreeEnergy.engines()]
engines_options.append("ALL")

node.addInput("FEP Engine", BSS.Gateway.String(help="Engine to run FEP with. BSS available engines, or ALL.",
                                             allowed=engines_options,
                                             default="ALL"))

node.addInput("LambdaWindows", BSS.Gateway.String(help="The number of lambda windows for regular transformations.",
                                             allowed=["3", "4", "5", "6", "7", "8", "9", "10", "11", "12", "13", "14", "15", "16", "17", "18", "19", "20"],
                                             default="11"))

# node.addInput("DiffLambdaWindows", BSS.Gateway.String(help="The number of lambda windows for difficult transformations.",
#                                              allowed=["4", "5", "6", "7", "8", "9", "10", "11", "12", "13", "14", "15", "16", "17", "18", "19", "20"],
#                                              default="17"))
                                             
# node.addInput("LOMAP Threshold", BSS.Gateway.String(help="The LOMAP score threshold to define difficult transformations.",
#                                              allowed=["0.1", "0.2", "0.3", "0.4", "0.5", "0.6", "0.7", "0.8", "0.9"],
#                                              default="0.4"))

node.addInput("Number of repeats", BSS.Gateway.String(help="The number of repeats of the simulation.",
                                             allowed=[str(i) for i in range (1,11)],
                                             default=str(3)))

node.addInput("Keep trajectories", BSS.Gateway.String(help="Whether to keep the trajectory files or not.",
                                             allowed=["None","0,0.5,1", "0,1", "All"],
                                             default="0,0.5,1"))

node.showControls()


In [ ]:
#nodes to pick things
node_ana = BSS.Gateway.Node("A node to specify the analysis for molecular dynamics simulation.")

node_ana.addInput("estimator", BSS.Gateway.String(help="Estimator used to analyse.",
                                             allowed=["MBAR", "TI"],
                                             default="MBAR"))

node_ana.addInput("method", BSS.Gateway.String(help="method used to analyse.",
                                             allowed=["alchemlyb", "native"],
                                             default="alchemlyb"))

node_ana.showControls()


##### <span style="color:teal">The FEP Network</span>  
<a id="perts"></a>


In [ ]:
#generate transformation network based on ligands
ligand_files = sorted(glob.glob(f"{path_to_ligands}/*.sdf"))

ligands = []
ligand_names = []

for filepath in ligand_files:
    # append the molecule object to a list.
    ligands.append(BSS.IO.readMolecules(filepath)[0])
    
    # append the molecule name to another list so that we can use the name of each molecule in our workflow.
    ligand_names.append(filepath.split("/")[-1].replace(".sdf",""))

print(ligand_names)
print(len(ligand_names))

In [ ]:
# # if wanna remove certain ligands
# ligs_remove_list = ['lig_ejm51', 'lig_ejm52', 'lig_ejm53']
ligs_remove_list = ['lig_ejm53']

for index, lig in enumerate(ligand_names):
    if lig in ligs_remove_list:
        ligand_names.remove(lig)
        del ligands[index]

print(ligand_names)
print(len(ligand_names))
print(len(ligands))

In [ ]:
transformations, lomap_scores = BSS.Align.generateNetwork(ligands, plot_network=True, names=ligand_names, work_dir=f"{exec_folder}/visualise_network_lomap")


In [ ]:
# dict of perts
pert_network_dict = {}
transformations_named = [(ligand_names[transf[0]], ligand_names[transf[1]]) for transf in transformations]
for score, transf in sorted(zip(lomap_scores, transformations_named)):
    pert_network_dict[transf] = score
    print(transf, score)

# for t, s in sorted(zip(transformations_named, lomap_scores), key=lambda x: x[1]):
#     print(f"({ligand_names[t[0]]}, {ligand_names[t[1]]}), {s}")

In [ ]:
# # alternatively, if we want to recalculate the LOMAP score for this perturbation:
# # if we consider the ligands list, we can use the index of the ligands we want to add an edge for.
# # first we need to choose our ligands.
# lig_a = "ejm48"
# lig_b = "ejm51"

# # then, we need to find this index for our chosen edge that we are adding.
# lig_a_index = ligand_names.index(lig_a)
# lig_b_index = ligand_names.index(lig_b)
# # finally, we need to calculate this single lomap score.
# single_transformation, single_lomap_score = BSS.Align.generateNetwork([ligands[lig_a_index], ligands[lig_b_index]], names=[ligand_names[lig_a_index], ligand_names[lig_b_index]], plot_network=False)
# print(f"LOMAP score for {lig_a} to {lig_b} is {single_lomap_score[0]} .")
# pert_network_dict[(lig_a, lig_b)] = single_lomap_score[0]

In [ ]:
# # answer
# for key in [('ejm42', 'ejm54'), ('ejm31', 'ejm45')]:
#     del pert_network_dict[key]
# # both the lowest scores are with ejm54, however if we remove these ejm54 won't be connected anymore, so we will remove
# # the next lowest perturbation instead.

Now that we have made these changes to the network, we want to visualise it again. We can do this using NetworkX.

In [ ]:
# Generate the graph.
graph = nx.Graph()

# Loop over the nligands and add as nodes to the graph.
for lig in ligand_names:
    graph.add_node(lig, label=lig, labelloc="t")

# Loop over the edges in the dictionary and add to the graph.
for edge in pert_network_dict:
    graph.add_edge(edge[0],edge[1],
                    label=(pert_network_dict[edge]))

# Plot the networkX graph.
pos = nx.kamada_kawai_layout(graph)
plt.figure(figsize=(14,14), dpi=150)
nx.draw(
    graph, pos, edge_color='black', width=1, linewidths=1,
    node_size=1500, node_color='skyblue', font_size = 12,
    labels={node: node for node in graph.nodes()})

nx.draw_networkx_edge_labels(
    graph, pos,
    edge_labels=pert_network_dict,
    font_color='purple', font_size=10)

plt.savefig(f"{exec_folder}/adjusted_lomap_network.png", dpi=300)
plt.show()

In [ ]:
# # adding an intermediate
# #generate transformation network based on ligands
# ligand_files_list = glob.glob(f"{path_to_ligands}/*.sdf")
# ligand_files_list.append(f"{path_to_ligands}/intermediate/intermediate_H.sdf")
# ligand_files = sorted(ligand_files_list)

# ligands = []
# ligand_names = []

# for filepath in ligand_files:
#     # append the molecule object to a list.
#     ligands.append(BSS.IO.readMolecules(filepath)[0])
    
#     # append the molecule name to another list so that we can use the name of each molecule in our workflow.
#     ligand_names.append(filepath.split("/")[-1].replace(".sdf",""))

# transformations, lomap_scores = BSS.Align.generateNetwork(ligands, plot_network=True, names=ligand_names)

# pert_network_dict = {}
# transformations_named = [(ligand_names[transf[0]], ligand_names[transf[1]]) for transf in transformations]
# for score, transf in sorted(zip(lomap_scores, transformations_named)):
#     pert_network_dict[transf] = score
#     print(transf, score)


In [ ]:
# write the dict for lomap to a file

with open(f"{exec_folder}/network_lomap_scores.dat", "w") as scores_file:
    writer = csv.writer(scores_file)

    for score, transf in sorted(zip(lomap_scores, transformations_named)):
        pert_network_dict[transf] = score
        writer.writerow([transf[0], transf[1], score])


##### <span style="color:teal">Preparing for the FEP pipeline</span>  


In [ ]:
# write ligands file.
with open(f"{exec_folder}/ligands.dat", "w") as ligands_file:
    writer = csv.writer(ligands_file)
    for lig in ligand_names:
        writer.writerow([lig])

# create protocol. 
protocol = [
    f"ligand forcefield = {node.getInput('Ligand FF')}",
    f"protein forcefield = {node.getInput('Protein FF')}",
    f"solvent = {node.getInput('Water Model')}",
    f"box edges = {node.getInput('Box Edges')}",
    f"box type = {node.getInput('Box Shape')}",
    f"protocol = default", # can remove this line? need to change all script lines too
    f"sampling = {node.getInput('Run Time')}",
    f"HMR = {node.getInput('HMR')}",
    f"repeats = {node.getInput('Number of repeats')}",
    f"keep trajectories = {node.getInput('Keep trajectories')}",
]


# write protocol to file.
with open(f"{exec_folder}/protocol.dat", "w") as protocol_file:
    writer = csv.writer(protocol_file)

    for prot_line in protocol:
        writer.writerow([prot_line])

# create an analysis protocol
ana_protocol = [
    f"estimator = {ana_node.getInput('estimator')}",
    f"method = {ana_node.getInput('method')}"
]

# write protocol to file.
with open(f"{exec_folder}/analysis_protocol.dat", "w") as protocol_file:
    writer = csv.writer(protocol_file)

    for prot_line in ana_protocol:
        writer.writerow([prot_line])

In [ ]:
# write perts file. Base the lambda schedule on the file generated in the previous cell.
np.set_printoptions(formatter={'float': '{: .4f}'.format})

# from protocol, derive the engine we want to use on the cluster.
engine = node.getInput('FEP Engine').upper()

with open(f"{exec_folder}/network_lomap.dat", "w") as network_file:

    writer = csv.writer(network_file, delimiter=" ")
    
    for pert, lomap_score in pert_network_dict.items():
        # # based on the provided (at top of notebook) lambda allocations and LOMAP threshold, decide allocation.
        # if lomap_score == None or lomap_score < float(node.getInput("LOMAP Threshold")):
        #     num_lambda = node.getInput("DiffLambdaWindows")
        # else:
        #     num_lambda = node.getInput("LambdaWindows")
        
        num_lambda = node.getInput("LambdaWindows") # same no lamdda windows for all
       
        # given the number of allocated lambda windows, generate an array for parsing downstream.
        lam_array_np = np.around(np.linspace(0, 1, int(num_lambda)), decimals=5)

        # make the array into a format readable by bash.
        lam_array = str(lam_array_np).replace("[ ", "").replace("]", "").replace("  ", ",").replace('\n', '')

        # write out both directions for this perturbation.
        if node.getInput("FEP Engine") == "ALL":
            for eng in BSS.FreeEnergy.engines():
                writer.writerow([pert[0], pert[1], len(lam_array_np), lam_array, eng])
        else:
            writer.writerow([pert[0], pert[1], len(lam_array_np), lam_array, node.getInput("FEP Engine")])
        # writer.writerow([pert[1], pert[0], len(lam_array_np), lam_array, engine])   

##### <span style="color:teal">Generating the RBFENN</span>  


In [ ]:
def scaleArray(arr):
    """Scales an array to be the inverse in the range [0-1]."""
    
    # normalise to the range 0-1.
    return minmax_scale(1 /  arr, feature_range=(0,1))


In [ ]:
# get the FEPNN SEM prediction per ligand.
perts = {}
for cats_file in glob.glob(f"{cats_files_path}/{tgt_to_run}_*"):
    
    with open(cats_file, "r") as readfile:
        reader = csv.reader(readfile)
        next(reader)
        for row in reader:
            pert = row[0]
            pred_sem = float(row[1])
            
            if not pert in perts:
                perts[pert] = [pred_sem]
            else:
                perts[pert].append(pred_sem)
            
# compute the mean SEM prediction per pert.
pert_names = []
pert_sems = []
for pert, sems in perts.items():
    mean_sem = np.mean(sems)
    pert_names.append(pert)
    pert_sems.append(float(mean_sem))

# now scale the sems to [0-1].
pert_sems = scaleArray(np.array(pert_sems))

for pert, val in zip(pert_names, pert_sems):
    perts[pert] = val

In [ ]:
written = []
with open(f"{exec_folder}/links_file.in", "w") as writefile:
    writer = csv.writer(writefile, delimiter =" ")
    
    for pert_name, value in perts.items():
        # find the lomap filename for both ligs.
        liga_lomap_name = None
        ligb_lomap_name = None
        for filename in glob.glob(f"{path_to_ligands}/*.sdf"):
            # if "lig_8" in filename:
            #     continue # exclude +1 ligands from tnks2 set.
            if pert_name.split("~")[0] in filename:
                liga_lomap_name = filename.split("/")[-1].split(".")[0]#.replace("ejm","ejm_").replace("jmc","jmc_")
            elif pert_name.split("~")[1] in filename:
                ligb_lomap_name = filename.split("/")[-1].split(".")[0]#.replace("ejm","ejm_").replace("jmc","jmc_")
            
            if liga_lomap_name and ligb_lomap_name:
                if not [liga_lomap_name, ligb_lomap_name] in written:
                    writer.writerow([liga_lomap_name, ligb_lomap_name, value])
                    
                    written.append([liga_lomap_name, ligb_lomap_name])

In [ ]:
# ligands and ligands_names already exists due to lomap above

transformations_fepnn, rbfenn_scores = BSS.Align.generateNetwork(ligands, plot_network=True, names=ligand_names, 
                                                         work_dir=f"{exec_folder}/visualise_network_rbfenn",
                                                         links_file=f"{exec_folder}/links_file.in"
                                                        )

In [ ]:
# dict of perts rbfenn
rbfenn_pert_network_dict = {}
transformations_named_rbfenn = [(ligand_names[transf[0]], ligand_names[transf[1]]) for transf in transformations_fepnn]
for score, transf in sorted(zip(rbfenn_scores, transformations_named_rbfenn)):
    rbfenn_pert_network_dict[transf] = score
    print(transf, score)

In [ ]:
# write the rbfenn to a different network file

with open(f"{exec_folder}/network_rbfenn.dat", "w") as network_file:

    writer = csv.writer(network_file, delimiter=" ")
    
    for pert, rbfenn_score in rbfenn_pert_network_dict.items():
        # # based on the provided (at top of notebook) lambda allocations and LOMAP threshold, decide allocation.
        # if rbfenn_score == None or rbfenn_score < float(node.getInput("LOMAP Threshold")):
        #     num_lambda = node.getInput("DiffLambdaWindows")
        # else:
        #     num_lambda = node.getInput("LambdaWindows")

        num_lambda = node.getInput("LambdaWindows") 
       
        # given the number of allocated lambda windows, generate an array for parsing downstream.
        lam_array_np = np.around(np.linspace(0, 1, int(num_lambda)), decimals=5)

        # make the array into a format readable by bash.
        lam_array = str(lam_array_np).replace("[ ", "").replace("]", "").replace("  ", ",").replace('\n', '')

        # write out both directions for this perturbation.
        if node.getInput("FEP Engine") == "ALL":
            for eng in BSS.FreeEnergy.engines():
                writer.writerow([pert[0], pert[1], len(lam_array_np), lam_array, eng])
        else:
            writer.writerow([pert[0], pert[1], len(lam_array_np), lam_array, node.getInput("FEP Engine")])
        # writer.writerow([pert[1], pert[0], len(lam_array_np), lam_array, engine])         

# write the dict for lomap to a file

with open(f"{exec_folder}/network_rbfenn_scores.dat", "w") as scores_file:
    writer = csv.writer(scores_file)

    for score, transf in sorted(zip(rbfenn_scores, transformations_named_rbfenn)):
        rbfenn_pert_network_dict[transf] = score
        writer.writerow([transf[0], transf[1], score])


In [ ]:
# Generate the graph.
graph = nx.Graph()

# Loop over the nligands and add as nodes to the graph.
for lig in ligand_names:
    graph.add_node(lig, label=lig, labelloc="t")

# Loop over the edges in the dictionary and add to the graph.
for edge in rbfenn_pert_network_dict:
    graph.add_edge(edge[0],edge[1],
                    label=(rbfenn_pert_network_dict[edge]))

# Plot the networkX graph.
pos = nx.kamada_kawai_layout(graph)
plt.figure(figsize=(14,14), dpi=150)
nx.draw(
    graph, pos, edge_color='black', width=1, linewidths=1,
    node_size=1500, node_color='skyblue', font_size = 12,
    labels={node: node for node in graph.nodes()})

nx.draw_networkx_edge_labels(
    graph, pos,
    edge_labels=rbfenn_pert_network_dict,
    font_color='purple', font_size=10)

plt.savefig(f"{exec_folder}/adjusted_rbfenn_network.png", dpi=300)
plt.show()

##### <span style="color:teal">Comparing lomap and the rbfenn</span>  


In [ ]:
# get a list of the perts in each and all together
perts_lomap = []
perts_rbfenn = []
perts = []

with open(f"{exec_folder}/network_rbfenn_scores.dat", "r") as fepnn_file, \
        open(f"{exec_folder}/network_lomap_scores.dat", "r") as lomap_file:
    reader_fepnn = csv.reader(fepnn_file)
    reader_lomap = csv.reader(lomap_file)
    
    for line in reader_fepnn:
        perts_rbfenn.append(f"{line[0]}~{line[1]}")
        perts.append(f"{line[0]}~{line[1]}")
    for line in reader_lomap:
        perts_lomap.append(f"{line[0]}~{line[1]}")
        perts.append(f"{line[0]}~{line[1]}")

# write a file that contains the combined perts, directions are distinct     
combined_perts = []
filtered_out = 0
for pert in perts:
    
    if not pert in combined_perts:
        combined_perts.append(pert)
    else:
        filtered_out += 1
print(f"Removed {filtered_out} duplicate perts between lomap and rbfenn to give {len(combined_perts)} combined perts.")

# write a file that contains the unique perts, 1 direction only.      
filtered_perts = []
filtered_out = 0
for pert in combined_perts:
    
    inv_pert = pert.split("~")[1]+"~"+pert.split("~")[0]
    
    if not pert in filtered_perts and not inv_pert in filtered_perts:
        filtered_perts.append(pert)
    else:
        filtered_out += 1
print(f"Removed {filtered_out} inverse perts to give {len(filtered_perts)} unique perts, one direction only.")

# get the perts that are unique to each
unique_perts = []
unique_out_lomap = 0
unique_out_rbfenn = 0
shared_out = 0

for pert in perts_lomap:
    
    inv_pert = pert.split("~")[1]+"~"+pert.split("~")[0]
    
    if not pert in perts_rbfenn and not inv_pert in perts_rbfenn:
        unique_perts.append((pert, "lomap"))
        unique_out_lomap += 1

for pert in perts_rbfenn:
    
    inv_pert = pert.split("~")[1]+"~"+pert.split("~")[0]
    
    if not pert in perts_lomap and not inv_pert in perts_lomap:
        unique_perts.append((pert, "rbfenn"))
        unique_out_rbfenn += 1
    
for pert in combined_perts:

    inv_pert = pert.split("~")[1]+"~"+pert.split("~")[0]

    if pert in perts_lomap or inv_pert in perts_lomap:
        if pert in perts_rbfenn or inv_pert in perts_rbfenn:
            unique_perts.append((pert, "shared"))
            shared_out += 1

       
print(f"There are {unique_out_lomap} pert(s) unique to lomap and {unique_out_rbfenn} pert(s) unique to rbfenn.")
print(f"There are {shared_out} pert(s) shared between lomap and rbfenn")

In [ ]:
with open(f"{exec_folder}/combined_perts.dat", "w") as writefile:
    writer = csv.writer(writefile)
    for pert in combined_perts:
        writer.writerow([pert])
print(f"Total number of combined perturbations: {len(combined_perts)}")

with open(f"{exec_folder}/filtered_perts.dat", "w") as writefile:
    writer = csv.writer(writefile)
    for pert in filtered_perts:
        writer.writerow([pert])
print(f"Total number of filtered perturbations: {len(filtered_perts)}")

# write a file for the different perts
with open(f"{exec_folder}/unique_perts.dat", "w") as writefile:
    writer = csv.writer(writefile)
    for pert in unique_perts:
        writer.writerow([pert[0],pert[1]])
print(f"Total number of unique perturbations: {len(unique_perts)} (lomap: {unique_out_lomap}, rbfenn: {unique_out_rbfenn})")
print(f"Total number of shared perturbations: {shared_out}")

In [ ]:
# make a dicitonary of the perts for plotting the nx graph
both_pert_networks_dict = {}

for pert in filtered_perts:

    inv_pert = pert.split("~")[1]+"~"+pert.split("~")[0]
    
    if pert in perts_lomap and pert in perts_rbfenn:
        both_pert_networks_dict[pert] = "both"
    elif inv_pert in perts_lomap and pert in perts_rbfenn:
        both_pert_networks_dict[pert] = "both"
    elif pert in perts_lomap and pert not in perts_rbfenn:
        both_pert_networks_dict[pert] = "lomap"
    elif inv_pert in perts_lomap and pert not in perts_rbfenn:
        both_pert_networks_dict[pert] = "lomap"
    elif pert not in perts_lomap and pert in perts_rbfenn:
        both_pert_networks_dict[pert] = "rbfenn"
    elif inv_pert not in perts_lomap and pert in perts_rbfenn:
        both_pert_networks_dict[pert] = "rbfenn"      

# create dict for images for the nx graph
image_dict = {}
# list files in inputs
input_files_for_image = sorted(os.listdir(f"{exec_folder}/visualise_network_lomap/inputs"))
for in_file in input_files_for_image:
    lig_name_list = in_file.split("_")[1:]
    lig_name = '_'.join(lig_name_list).split(".")[0]
    lig_number = in_file.split("_")[0]
    image_dict[lig_name] = lig_number   

In [ ]:
both_pert_networks_dict

In [ ]:
# Generate the graph.
graph = nx.Graph()

# Loop over the nligands and add as nodes to the graph.
for lig in ligand_names:
    img = f"{exec_folder}/visualise_network_lomap/images/{image_dict[lig]}.png"
    graph.add_node(lig, image=img, label=lig, labelloc="t")

# make a dict of colours
# navy teal  #CC00CC
# clear is '#FF000000' 
colour_dict = {"both":'navy' ,"lomap":'teal' ,"rbfenn":'hotpink' }

# Loop over the edges in the dictionary and add to the graph.
for edge in both_pert_networks_dict:
    graph.add_edge(edge.split("~")[0],edge.split("~")[1],
                    color=colour_dict[both_pert_networks_dict[edge]]
                    )

# Plot the networkX graph.
pos = nx.kamada_kawai_layout(graph)
colours = nx.get_edge_attributes(graph,'color').values()

plt.figure(figsize=(12,12), dpi=150)
nx.draw(
    graph, pos, edge_color=colours, width=1, linewidths=5,
    node_size=2000, node_color='skyblue', font_size = 12,
    labels={node: node for node in graph.nodes()})

plt.savefig(f"{exec_folder}/compared_networks_no_images.png", dpi=300)
# plt.show()

# Convert to a dot graph.
dot_graph = nx.drawing.nx_pydot.to_pydot(graph)

# Write to a PNG.
network_plot = f"{exec_folder}/compared_networks.png"
dot_graph.write_png(network_plot)

# Create a plot of the network.
img = mpimg.imread(network_plot)
plt.figure(figsize=(10, 10))
plt.axis("off")
plt.imshow(img)


In [ ]:
# calculate the lomap score for the combined network file (unfiltered)
combined_pert_network_dict = {}

for pert in combined_perts:
    lig_a = pert.split("~")[0]
    lig_b = pert.split("~")[1]
    # then, we need to find this index for our chosen edge that we are adding.
    lig_a_index = ligand_names.index(lig_a)
    lig_b_index = ligand_names.index(lig_b)
    # finally, we need to calculate this single lomap score.
    single_transformation, single_lomap_score = BSS.Align.generateNetwork([ligands[lig_a_index], ligands[lig_b_index]], names=[ligand_names[lig_a_index], ligand_names[lig_b_index]], plot_network=False)
    print(f"LOMAP score for {lig_a} to {lig_b} is {single_lomap_score[0]} .")
    combined_pert_network_dict[(lig_a, lig_b)] = single_lomap_score[0]

In [ ]:
# write the combined to a different network file

with open(f"{exec_folder}/network_combined.dat", "w") as network_file:

    writer = csv.writer(network_file, delimiter=" ")
    
    for pert, score in combined_pert_network_dict.items():
        # # based on the provided (at top of notebook) lambda allocations and LOMAP threshold, decide allocation.
        # if score == None or score < float(node.getInput("LOMAP Threshold")):
        #     num_lambda = node.getInput("DiffLambdaWindows")
        # else:
        #     num_lambda = node.getInput("LambdaWindows")
        
        num_lambda = node.getInput("LambdaWindows")            
       
        # given the number of allocated lambda windows, generate an array for parsing downstream.
        lam_array_np = np.around(np.linspace(0, 1, int(num_lambda)), decimals=5)

        # make the array into a format readable by bash.
        lam_array = str(lam_array_np).replace("[ ", "").replace("]", "").replace("  ", ",").replace('\n', '')

        # write out both directions for this perturbation.
        if engine == "ALL":
            for eng in BSS.FreeEnergy.engines():
                writer.writerow([pert[0], pert[1], len(lam_array_np), lam_array, eng])
        else:
            writer.writerow([pert[0], pert[1], len(lam_array_np), lam_array, engine])
        # writer.writerow([pert[1], pert[0], len(lam_array_np), lam_array, engine])         